# Sequential Parameter Optimization
## `river` Hyperparameter Tuning: HTR with User Data 

In [1]:
pip list | grep spot

pyspot                                    0.0.1
pyspotriver                               0.0.3
pyspotstream                              0.2.0
spotPython                                0.0.6
spotRiver                                 0.0.8
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import sys
# !{sys.executable} -m pip install --upgrade build
# !{sys.executable} -m pip install --upgrade --force-reinstall http://www.gm.fh-koeln.de/~bartz/site/spotPython.tar.gz

## Example 1: HTR Hyperparameter

In [3]:
import numpy as np
from math import inf
from spotRiver.fun.hyperriver import HyperRiver
from spotPython.spot import spot
from spotRiver import data
from scipy.optimize import shgo
from scipy.optimize import direct
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt

### 1 The Objective Function 

* Here we will use the river `HATR` function.
* First, the function will be tested independently from `Spot`.

In [4]:
import numpy as np
import datetime as dt
from river import datasets
from river import time_series
from river import utils
import calendar
import math
from river import compose
from river import linear_model
from river import optim
from river import preprocessing
from river import metrics
from spotRiver.fun.hyperriver import HyperRiver

fun = HyperRiver(123).fun_HTR_iter_progressive
var_type = ["int"] * 2 + ["num"] * 2 + ["factor"]



### 2 The `fun_snarimax` from `spotPython`'s `HyperRiver` class

* Next, we use the implementation of the function from the `HyperRiver` class.
* Twelve hyperparameters to be optimized. Here:

0. `p` (int):
    Order of the autoregressive part.
    This is the number of past target values that will be included as features.
1. `d` (int):
    Differencing order.
2. `q` (int):
    Order of the moving average part.
    This is the number of past error terms that will be included as features.
3. `m` (int):
    Season length used for extracting seasonal features.
    If you believe your data has a seasonal pattern, then set this accordingly.
    For instance, if the data seems to exhibit a yearly seasonality,
    and that your data is spaced by month, then you should set this to `12`.
    Note that for this parameter to have any impact you should also set
    at least one of the `p`, `d`, and `q` parameters.
4. `sp` (int):
    Seasonal order of the autoregressive part.
    This is the number of past target values that will be included as features.
5. `sd` (int):
    Seasonal differencing order.
6. `sq`(int):
    Seasonal order of the moving average part.
    This is the number of past error terms that will be included as features.
7. `lr` (float):
    learn rate of the linear regression model. A river `preprocessing.StandardScaler`
    piped with a river `linear_model.LinearRegression` will be used.
8. `intercept_lr` (float): intercept of the the linear regression model.
    A river `preprocessing.StandardScaler` piped with a river `linear_model.LinearRegression`
    will be used.
9. `hour` (bool): If `True`, an hourly component is added.
10. `weekday` (bool): If `True`, an weekday component is added.
11. `month` (bool): If `True`, an monthly component is added.

* Note: Instead of using the Airline Passenger data as in notebook 01, we demonstrate the usage of user specified data which is *not* part of the `spotRiver` package!
* Remember: Previously, we used the setting, which will be overwritten in the following steps.

In [5]:
#                 p, d, q, m, sp, sd, sq, lr, intercept_lr, hour, weekday, month:
# lower = np.array([0, 0, 0, 1, 0,  0,  0,   0.00225, 0.05,   0,        0,   0])
# upper = np.array([1, 1, 1, 24, 1,  1,   1,  0.0025, 0.1,    0,        0,   1])
# fun_control = {"horizon": 12,
#                "grace_period": None,
#                "data": dataset,
#                }

## User Specified Data

In [6]:
# from pyspotstream.datasets import fetch_opm
# ds = fetch_opm(include_categorical=False, data_home="data", return_X_y=True)

* Important: Specify the location of your data, e.g., `/home/data/` etc.

In [7]:
from spotRiver.data.generic import GenericData
dataset = GenericData(filename="opm_cat.csv",
                      directory=".",
                      target="Sale Amount",
                      n_features=14,
                      n_samples=195_832,
                      converters={'List Year': int,
                                  'Assessed Value': float,
                                  'Sale Amount': float,
                                  'Sales Ratio': float,
                                  'lon': float,
                                  'lat': float,
                                  'timestamp_rec': float},
                      parse_dates=None
                      # parse_dates={"Date Recorded": "%Y-%m-%d"}
                      )


In [8]:
for x,y in dataset:
    print(x,y)
    break

{'List Year': 2001, 'Assessed Value': 107530.0, 'Sales Ratio': 0.575026738, 'lat': 41.50021401170393, 'lon': -72.87396263711757, 'timestamp_rec': 1001894400.0, 'Town': 'Bristol', 'Address': "MULTI #'S CONSTANCE LN", 'Property Type': 'Unknown', 'Residential Type': 'Unknown', 'Non Use Code': 'Unknown', 'Assessor Remarks': 'Unknown', 'OPM remarks': 'Unknown'} 187000.0


In [9]:
#
lower = np.array([100, 2,  1e-8, 0.025, 0])
upper = np.array([200, 10, 1e-6, 0.075, 2])
fun_control = {"data": dataset}

### 3 Run the `Spot` Optimizer

* Since the data is larger than the airline passengers data, the max. time is increased.

In [10]:
spot_htr = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   fun_repeats = 1,
                   max_time = 1,
                   noise = False,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type=var_type,
                   infill_criterion = "y",
                   n_points = 1,
                   seed=123,
                   log_level = 50,
                   show_models= False,
                   show_progress= True,
                   fun_control = fun_control,
                   design_control={"init_size": 10,
                                   "repeats": 1},
                   surrogate_control={"noise": False,
                                      "cod_type": "norm",
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": 5,
                                      "model_optimizer": differential_evolution,
                                      "model_fun_evals": 1000,
                                      })
spot_htr.run()

spotPython tuning: [##########] 100.00% Done...
spotPython tuning: [##########] 100.00% Done...
spotPython tuning: [##########] 100.00% Done...
spotPython tuning: [##########] 100.00% Done...
spotPython tuning: [##########] 100.00% Done...


### 4 Results 

* Print the Results

In [ ]:
spot_htr.print_results()

* Show the Progress

In [ ]:
spot_htr.plot_progress(log_y=True)

In [ ]:
spot_htr.print_importance()

In [ ]:
n = spot_htr.k
for i in range(n-1):
    for j in range(i+1, n):
        spot_htr.plot_contour(i=i, j=j)

## Build and Evaluate SNARIMAX Model with Tuned Hyperparameters

In [ ]:
X = spot_htr.to_all_dim(spot_htr.min_X.reshape(1,-1))
print(X)
p = X[:, 0]
d = X[:, 1]
q = X[:, 2]
m = X[:, 3]
sp = X[:, 4]
sd = X[:, 5]
sq = X[:, 6]
lr = X[:, 7]
intercept_lr = X[:, 8]
hour = X[:, 9]
weekday = X[:,10]
month = X[:,11]

In [ ]:
from spotRiver.utils.features import get_weekday_distances
from spotRiver.utils.features import get_ordinal_date
from spotRiver.utils.features import get_month_distances
from spotRiver.utils.features import get_hour_distances
h_i = int(hour)
w_i = int(weekday)
m_i = int(month)
# baseline:
extract_features = compose.TransformerUnion(get_ordinal_date)
if h_i:
    extract_features = compose.TransformerUnion(get_ordinal_date,
                                                get_hour_distances)
if w_i:
    extract_features = compose.TransformerUnion(extract_features,
                                                get_weekday_distances)
if m_i:
    extract_features = compose.TransformerUnion(extract_features,
                                                get_month_distances)
model = compose.Pipeline(
    ("extraction", extract_features),
    ("ts", time_series.SNARIMAX(
        p=int(p),
        d=int(d),
        q=int(q),
        m=int(m),
        sp=int(sp),
        sd=int(sd),
        sq=int(sq),
        regressor=compose.Pipeline(
            ("prep", preprocessing.StandardScaler()),
            ("lm", linear_model.LinearRegression(
                intercept_init=0,
                optimizer=optim.SGD(float(lr)),
                intercept_lr=float(intercept_lr),
            )),
        ),
    )),
)
# eval
metric = metrics.MAE()
res = time_series.evaluate(fun_control["data"], model, metric, horizon=fun_control["horizon"])

In [ ]:
y = res.metrics
z = 0.0
for j in range(len(y)):
    z = z + y[j].get()
z / len(y)

In [ ]:
model

In [ ]:
model["ts"].regressor["lm"].weights

In [ ]:
forecast = model.forecast(horizon=fun_control["horizon"])
forecast
